In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import re
import matplotlib.pyplot as plt
import seaborn as sns



def preprocess_energy_data(file_path):
    """
    Complete preprocessing pipeline for the energy consumption dataset

    Parameters:
    file_path (str): Path to the energy consumption CSV file
    """

    # Define expected columns and their types
    numeric_columns = [
        'population',
        'gdp_per_capita',
        'energy_consumption_per_capita',
        'carbon_emission',
        'renewable_energy_share',
        'electricity_price'
    ]

    categorical_columns = [
        'country',
        'region',
        'energy_policy',
        'climate_agreement',
        'urbanization',
        'energy_source',
        'income_group'
    ]

    # Expected ranges for numeric variables
    value_ranges = {
        'population': (1000000, 300000000),
        'gdp_per_capita': (1000, 60000),
        'energy_consumption_per_capita': (1, 10),
        'renewable_energy_share': (10, 90),
        'electricity_price': (0.05, 0.30)
    }

    # Updated valid categories with standardized income groups
    valid_categories = {
        'region': ['north america', 'europe', 'asia', 'south america', 'africa', 'oceania'],
        'energy_policy': ['strict', 'moderate', 'lenient'],
        'climate_agreement': ['paris', 'kyoto', 'none'],
        'urbanization': ['high', 'medium', 'low'],
        'energy_source': ['coal', 'natural gas', 'renewables', 'nuclear', 'oil'],
        'income_group': ['high', 'upper-middle', 'lower-middle', 'low']
    }

    # Income group standardization mapping
    income_group_mapping = {
        'high': 'high',
        'high income': 'high',
        'upper middle': 'upper-middle',
        'upper-middle': 'upper-middle',
        'upper middle income': 'upper-middle',
        'lower middle': 'lower-middle',
        'lower-middle': 'lower-middle',
        'lower middle income': 'lower-middle',
        'low': 'low',
        'low income': 'low'
    }

    try:
        # 1. Try different delimiters
        delimiters = [',', ';', '\t', '|']
        df = None
        for delimiter in delimiters:
            try:
                df = pd.read_csv(file_path, delimiter=delimiter)
                if len(df.columns) > 1:
                    print(f"Successfully read file with delimiter: '{delimiter}'")
                    break
            except:
                continue

        if df is None:
            raise ValueError("Could not read the file with any of the attempted delimiters")

        # If we still have only one column, try to split it
        if len(df.columns) == 1:
            print("Attempting to fix single-column data...")
            first_col = df.columns[0]
            for delimiter in delimiters:
                sample = df[first_col].iloc[0]
                if delimiter in sample:
                    df = pd.DataFrame([x.split(delimiter) for x in df[first_col].values.tolist()])
                    print(f"Split single column using delimiter: '{delimiter}'")
                    break

        print("Initial data shape:", df.shape)

        # 2. If we have the correct number of columns but no headers
        if len(df.columns) == 14 and all(str(x).isnumeric() for x in df.columns):
            expected_columns = ['year', 'country', 'population', 'gdp_per_capita',
                                'energy_consumption_per_capita', 'carbon_emission',
                                'renewable_energy_share', 'electricity_price', 'region',
                                'energy_policy', 'climate_agreement', 'urbanization',
                                'energy_source', 'income_group']
            df.columns = expected_columns

        # 3. Clean column names
        df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

        # 4. Remove duplicates
        df = df.drop_duplicates().reset_index(drop=True)

        # 5. Process year
        if 'year' not in df.columns:
            raise ValueError("Year column not found in dataset")

        df['year'] = pd.to_numeric(df['year'], errors='coerce')
        df = df[df['year'].between(2000, 2023)].copy()

        # 6. Process numeric columns
        for col in numeric_columns:
            if col in df.columns:
                # Clean and convert to numeric
                if df[col].dtype == 'object':
                    df[col] = df[col].str.replace(r'[^\d.-]', '', regex=True)
                df[col] = pd.to_numeric(df[col], errors='coerce')

                # Handle ranges
                if col in value_ranges:
                    min_val, max_val = value_ranges[col]
                    df[col] = df[col].clip(min_val, max_val)

                # Handle outliers
                Q1 = df[col].quantile(0.25)
                Q3 = df[col].quantile(0.75)
                IQR = Q3 - Q1
                lower_bound = Q1 - 1.5 * IQR
                upper_bound = Q3 + 1.5 * IQR

                mask = (df[col] < lower_bound) | (df[col] > upper_bound)
                if mask.any():
                    print(f"\nReplacing {mask.sum()} statistical outliers in {col}")
                    df.loc[mask, col] = df[col].median()

        # 7. Process categorical columns with special handling for income_group
        for col in categorical_columns:
            if col in df.columns:
                # Clean categorical values
                df[col] = df[col].astype(str).str.lower().str.strip()
                df[col] = df[col].str.replace(r'[^\w\s-]', '', regex=True)

                # Special handling for income_group
                if col == 'income_group':
                    # First try exact mapping
                    df[col] = df[col].map(income_group_mapping)

                    # For any unmapped values, try to find the closest match
                    unmapped = df[col].isna()
                    if unmapped.any():
                        print(f"\nFound non-standard income group categories:")
                        print(df.loc[unmapped, col].unique())
                        df.loc[unmapped, col] = df[col].mode().iloc[0]

                elif col in valid_categories:
                    mask = ~df[col].isin(valid_categories[col])
                    if mask.any():
                        print(f"\nFound invalid categories in {col}:")
                        print(df.loc[mask, col].unique())
                        df.loc[mask, col] = df[col].mode().iloc[0]

        # 8. Handle missing values
        missing_percentages = df.isnull().mean() * 100

        # Drop columns with too many missing values (>65%)
        cols_to_drop = missing_percentages[missing_percentages > 65].index
        if len(cols_to_drop) > 0:
            print(f"\nDropping columns with >65% missing values: {', '.join(cols_to_drop)}")
            df = df.drop(columns=cols_to_drop)

        # Fill remaining missing values
        for col in df.columns:
            if df[col].isnull().any():
                if col in numeric_columns:
                    df[col] = df[col].fillna(df[col].median())
                else:
                    df[col] = df[col].fillna(df[col].mode().iloc[0])

        # 9. Set final data types
        for col in numeric_columns:
            if col in df.columns:
                df[col] = df[col].astype(float)

        for col in categorical_columns:
            if col in df.columns:
                df[col] = df[col].astype('category')

        df['year'] = df['year'].astype(int)

        # 10. Generate summary
        print("\nFinal data shape:", df.shape)
        print("\nIncome Group distribution:")
        print(df['income_group'].value_counts())

        print("\nNumerical columns summary:")
        print(df[numeric_columns].describe())

        print("\nCategorical columns summary:")
        for col in categorical_columns:
            if col in df.columns:
                print(f"\n{col} value counts:")
                print(df[col].value_counts().head())

        # 11. Export cleaned data
        output_path = file_path.rsplit('.', 1)[0] + '_cleaned.csv'
        df.to_csv(output_path, index=False)
        print(f"\nCleaned data exported to: {output_path}")

        return df

    except Exception as e:
        print(f"Error processing data: {str(e)}")
        print("\nDetailed error information:")
        if 'df' in locals():
            print("Current dataframe shape:", df.shape)
            print("Current columns:", df.columns.tolist())
            print("First few rows:")
            print(df.head())
        return None

preprocess_energy_data(r"/content/Dirty_Energy_Consumption_Dataset.csv")

In [ ]:
# Mehmet Toprak Özen
df = preprocess_energy_data(r"/content/Dirty_Energy_Consumption_Dataset.csv") # Call the function and assign the result to df

# Bar Chart for Average Electricity Price by Urbanization Level and Income Group
avg_price = df.groupby(['income_group', 'urbanization'])['electricity_price'].mean().reset_index()
sns.barplot(data=avg_price, x='income_group', y='electricity_price', hue='urbanization', palette="Blues")
plt.title('Average Electricity Price by Income Group and Urbanization Level')
plt.xlabel('Income Group')
plt.ylabel('Average Electricity Price (USD per kWh)')
plt.legend(title='Urbanization Level', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

# Box Plot for Energy Source, Electricity Price, and Carbon Emission
palette_toprak = sns.color_palette("Blues", n_colors=df['region'].nunique())
sns.boxplot(data=df, x='energy_source', y='electricity_price', hue='region', palette=palette_toprak)
plt.title('Electricity Price by Energy Source and Region')
plt.xlabel('Energy Source')
plt.ylabel('Electricity Price (USD per kWh)')
plt.xticks(rotation=45)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', title="Region")
plt.show()

# 2. Violin Plot for Electricity Price by Energy Policy
ax = sns.violinplot(data=df, x='energy_policy', y='electricity_price',
                        scale='width', inner='quartile', palette="Blues")
plt.title('Electricity Price by Energy Policy')
plt.xlabel('Energy Policy')
plt.ylabel('Electricity Price (USD per kWh)')
plt.xticks(rotation=45)

    # Manually create legend entries
handles = [plt.Rectangle((0, 0), 1, 1, fc=color)
            for color in sns.color_palette("Blues", n_colors=len(df['energy_policy'].unique()))]

    # Add the legend
ax.legend(handles, df['energy_policy'].unique(), title="Energy Policy",
          bbox_to_anchor=(1.05, 1), loc='upper left')

plt.show()


In [ ]:
# Ahmet Furkan Köşedaşı
df=pd.read_csv("C:/Users/aahme/PycharmProjects/pythonProject/Cleaned_Energy_Consumption_Dataset_cleaned.csv")

def analyse_energy_data(df):
# How do countries' participation in climate agreements and their energy policies correlate with changes in carbon emissions?

    order = ['lenient', 'moderate', 'strict']
    agreement_names = {"kyoto": "Kyoto", "none": "No Agreement", "paris": "Paris"}
    df['climate_agreement'] = df['climate_agreement'].replace(agreement_names)
    plt.figure(figsize=(12, 8))

    sns.boxplot(x='climate_agreement',
                y='carbon_emission',
                hue='energy_policy',
                data=df,
                palette="Blues",
                hue_order=order)

    plt.title('Carbon Emissions by Climate Agreement and Energy Policy', fontsize=16, pad=20)
    plt.xlabel('Climate Agreement', fontsize=12)
    plt.ylabel('Carbon Emissions', fontsize=12)

    plt.xticks(rotation=45)

    plt.legend(title='Energy Policy', bbox_to_anchor=(1.05, 1), loc='upper left')

    plt.tight_layout()

    plt.show()

analyse_energy_data(df)

In [ ]:
#Arda Ergüven
data = pd.read_csv("/content/Cleaned_Energy_Consumption_Dataset_cleaned.csv")

custom_palette = sns.color_palette(["#ffcc00", "#ffdd44", "#1f77b4", "#4c78a8", "#2ca02c", "#68c18e"])

plt.figure(figsize=(14, 8))
sns.violinplot(data=data, x='energy_source', y='carbon_emission', hue='region', split=True, palette=custom_palette)
plt.title('Carbon Emissions Distribution by Energy Source and Region', fontsize=16)
plt.xlabel('Energy Source', fontsize=14)
plt.ylabel('Carbon Emissions', fontsize=14)
plt.legend(title='Region', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(axis='y')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Davut Varlık
df = pd.read_csv('cleanedData.csv')

plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='energy_consumption_per_capita', y='gdp_per_capita',
                hue='climate_agreement', size='renewable_energy_share',
                sizes=(20, 200), alpha=0.7, legend=False)

plt.title('Relation between Energy Counsumption, GDP and Renewable Energy Share')  # Set the title
plt.xlabel('Energy Consumption Per Capita (MWh)')  # Label the x axis
plt.ylabel('GDP per person $')  # Label the y axis
plt.show()


#Table code
grouped_data = df.groupby('climate_agreement') # Group the data by climate agreement status

# Calculate the mean values for each group
average_values = grouped_data[['energy_consumption_per_capita', 'gdp_per_capita', 'renewable_energy_share']].mean()

# Round the values to two decimal places
average_values = average_values.round(2)

display(average_values)

In [ ]:
# Tunahan Turgut
# Years into groups of 4
df['year_group'] = pd.cut(
    df['year'],
    bins=range(2000, 2028, 4),  # To make intervals by years
    labels=["2000-2003", "2004-2007", "2008-20011", "2012-2015",
            "2016-2019", "2020-2023"],
    right=False,  # To arrange intervals correctly
)


# Plot the heatmap
plt.figure(figsize=(15, 10))  # Adjust figure size for better visibility
sns.heatmap(
    pd_crosstab_grouped,
    annot=True,    # Set True to see numeric annotations on heatmap
    fmt=".1f",      # Format for annotations
    cmap='YlGnBu',  # Chose proper (common) color for heatmap
    cbar_kws={'label': 'Energy Consumption Per Capita(MWh)'},  # Color bar label
    linewidths=0.5  # Add lines between cells for better readability
)

# Creating table and assign variables
pd_crosstab_grouped = pd.pivot_table(
    df,
    values='energy_consumption_per_capita',
    index='region',
    columns='year_group',
    aggfunc='mean'
)

# Customize axis labels and title
plt.title("Energy Consumption per Capita in Regions According to Time Intervals",
          fontsize=20, fontweight='bold', pad=20)
plt.xlabel("Time Interval", fontsize=16, fontweight='bold')
df['region'] = df['region'].str.upper()
plt.ylabel("Region", fontsize=16, fontweight='bold' )
#fontweight='bold' is used in order to make headlines stand out

# Rotate tick marks for visibility
plt.xticks(rotation=45, fontsize=12)  # Slight tilt for better visibility
plt.yticks(rotation=0, fontsize=12)

plt.tight_layout()  # Adjust layout to prevent overlapping
plt.show()

In [ ]:
#Bilginalp Büyüktaş
data = pd.read_csv('/content/cleanedData.csv')

plt.figure(figsize=(18, 6))

sns.histplot(data=data.query("urbanization == 'low'"), x='renewable_energy_share', kde=True, bins=20, color="#ffdd44", alpha=0.6, ax=plt.subplot(1, 3, 1))
plt.title('Low Urbanization')
plt.xlabel('Renewable Energy (%)')
plt.grid(axis='y')

sns.histplot(data=data.query("urbanization == 'medium'"), x='renewable_energy_share', kde=True, bins=20, color="#1f77b4", alpha=0.6, ax=plt.subplot(1, 3, 2))
plt.title('Medium Urbanization')
plt.xlabel('Renewable Energy (%)')
plt.grid(axis='y')

sns.histplot(data=data.query("urbanization == 'high'"), x='renewable_energy_share', kde=True, bins=20, color="#68c18e", alpha=0.6, ax=plt.subplot(1, 3, 3))
plt.title('High Urbanization')
plt.xlabel('Renewable Energy (%)')
plt.grid(axis='y')

plt.suptitle('Renewable Energy Share by Urbanization Levels', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# Sertaç Kandemir
data = pd.read_csv(r"C:\Users\Sertac\Desktop\python Projects\Dirty_Energy_Consumption_Dataset_cleaned.csv")
income_order = ['high', 'upper-middle', 'lower-middle', 'low']
income_energy = data.groupby('income_group')['energy_consumption_per_capita'].mean().reset_index()
income_trends = data.groupby(['year', 'income_group'])['energy_consumption_per_capita'].mean().reset_index()
sns.set_theme(style="whitegrid")
blue_palette = sns.light_palette("steelblue", n_colors=4, reverse=True)
plt.figure(figsize=(12, 6))
ax = sns.barplot(data=income_energy,
                 x='income_group',
                 y='energy_consumption_per_capita',
                 palette=blue_palette,
                 order=income_order)
for i in ax.containers:
    ax.bar_label(i, fmt='%.2f', padding=3)
plt.title('Average Energy Consumption per Capita by Income Group',
          fontsize=14,
          pad=20,
          color='darkblue')
plt.xlabel('Income Group', fontsize=12, color='navy')
plt.ylabel('Average Energy Consumption per Capita (Gigajoules)', fontsize=12, color='navy')
plt.grid(axis='y', linestyle='--', alpha=0.3)
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()
plt.figure(figsize=(12, 6))
sns.boxplot(data=data,
            x='income_group',
            y='energy_consumption_per_capita',
            palette=sns.light_palette("navy", n_colors=4, reverse=True),
            order=income_order)
plt.title('Distribution of Energy Consumption per Capita by Income Group',
          fontsize=14,
          color='darkblue')
plt.xlabel('Income Group', fontsize=12, color='navy')
plt.ylabel('Energy Consumption per Capita (Gigajoules)', fontsize=12, color='navy')
plt.show()
max_year = income_trends['year'].max()
last_10_years = range(max_year - 9, max_year + 1)
filtered_trends = income_trends[income_trends['year'].isin(last_10_years)]
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Energy Consumption Trends by Income Group (2014-2023)',
             fontsize=16,
             color='darkblue',
             y=1.02)
axes = axes.flatten()
for idx, income in enumerate(income_order):
    data_subset = filtered_trends[filtered_trends['income_group'] == income]
    sns.regplot(data=data_subset,
                x='year',
                y='energy_consumption_per_capita',
                color='steelblue',
                scatter_kws={'s': 50, 'alpha': 0.5},
                line_kws={'color': 'navy'},
                ax=axes[idx])
    axes[idx].set_title(f'{income.capitalize()} Income Countries',
                        fontsize=12,
                        color='navy',
                        pad=10)
    axes[idx].set_xlabel('Year', fontsize=10, color='navy')
    axes[idx].set_ylabel('Energy Consumption\n(Gigajoules per Capita)',
                         fontsize=10,
                         color='navy')
    axes[idx].grid(True, linestyle='--', alpha=0.3)
    axes[idx].set_xticks([year for year in last_10_years if year % 2 == 0])
    axes[idx].tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.show()